<a href="https://colab.research.google.com/github/adalbertii/modele-NLP/blob/main/NLP_english_spanish_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-nlp --upgrade
!pip install rouge-score # zestaw wskaźników i pakiet oprogramowania używany do oceny modeli
                         # do automatycznego podsumowania i tłumaczenia maszynowego w przetwarzaniu języka naturalnego

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 31.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=262afc5fd6753164ea41ee6ff98ed86e42857abd6fcc0c5a091d331d0d2a0150
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import keras_nlp
#---------------------------------------
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import pathlib
import random
import string
import re
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import sklearn
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

Using TensorFlow backend


In [3]:
class Config:
    vocab_size = 15000 # rozmiar słownika
    sequence_length = 20
    batch_size = 512
    validation_split = 0.15
    embed_dim = 256
    latent_dim = 256
    num_heads = 2
    epochs = 30 # liczba epko w procesie trenowania
    start_token = "[start]"
    end_token = "[end]"

config = Config()

In [ ]:
/content/drive/MyDrive/dane/data-english-spanish.csv

In [4]:
data = pd.read_csv("/content/drive/MyDrive/dane/data-english-spanish.csv")
data.head()

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [5]:
data[100:110]

,english,spanish
100,Go away!,¡Salga de aquí!
101,Go away!,¡Largo!
102,Go away!,¡Vete ya!
103,Go away!,¡Órale!
104,Go away!,¡A la calle!
105,Go away.,¡Vete de aquí!
106,Go away.,¡Lárgate!
107,Go away.,¡Largo!
108,Go away.,¡Vete ya!
109,Go away.,¡Órale!


In [7]:
data["spanish"] = data["spanish"].apply(lambda item: f"{config.start_token} " + item + f" {config.end_token}")

In [8]:
data["spanish"][100:110]

100    [start] [start] ¡Salga de aquí! [end] [end]
101            [start] [start] ¡Largo! [end] [end]
102          [start] [start] ¡Vete ya! [end] [end]
103            [start] [start] ¡Órale! [end] [end]
104       [start] [start] ¡A la calle! [end] [end]
105     [start] [start] ¡Vete de aquí! [end] [end]
106          [start] [start] ¡Lárgate! [end] [end]
107            [start] [start] ¡Largo! [end] [end]
108          [start] [start] ¡Vete ya! [end] [end]
109            [start] [start] ¡Órale! [end] [end]
Name: spanish, dtype: object

In [9]:
data.head()

,english,spanish
0,Go.,[start] [start] Ve. [end] [end]
1,Go.,[start] [start] Vete. [end] [end]
2,Go.,[start] [start] Vaya. [end] [end]
3,Go.,[start] [start] Váyase. [end] [end]
4,Hi.,[start] [start] Hola. [end] [end]


In [20]:
data.shape

(118964, 2)

In [10]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
print(strip_chars)

!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


In [11]:
def spanish_standardize(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]"%re.escape(strip_chars), "")

In [12]:
english_vectorization = TextVectorization(
    max_tokens=config.vocab_size,
    output_mode="int",
    output_sequence_length=config.sequence_length,
)

In [13]:
spanish_vectorization = TextVectorization(
    max_tokens=config.vocab_size,
    output_mode="int",
    output_sequence_length=config.sequence_length + 1,
    standardize=spanish_standardize,
)

In [14]:
english_vectorization.adapt(list(data["english"]))

In [15]:
spanish_vectorization.adapt(list(data["spanish"]))

In [21]:
def preprocess(english, spanish):
    english = english_vectorization(english)
    spanish = spanish_vectorization(spanish)
    return ({"encoder_inputs": english, "decoder_inputs": spanish[:, :-1]}, spanish[:, 1:])


In [22]:
def make_dataset(df, batch_size, mode):
    dataset = tf.data.Dataset.from_tensor_slices((list(df["english"]), list(df["spanish"])))
    if mode == "train":
       dataset = dataset.shuffle(batch_size * 4)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(preprocess)
    dataset = dataset.prefetch(tf.data.AUTOTUNE).cache()
    return dataset

In [23]:
train, valid = train_test_split(data, test_size=config.validation_split)
train.shape, valid.shape

((101119, 2), (17845, 2))

In [24]:
train_ds = make_dataset(train, batch_size=config.batch_size, mode="train")
valid_ds = make_dataset(valid, batch_size=config.batch_size, mode="valid")

In [25]:
type(train_ds)

tensorflow.python.data.ops.cache_op.CacheDataset

In [26]:
for batch in train_ds.take(1):
    print(batch)

({'encoder_inputs': <tf.Tensor: shape=(512, 20), dtype=int64, numpy=
array([[  21,    8, 2718, ...,    0,    0,    0],
       [   9,   44,  794, ...,    0,    0,    0],
       [  49, 5145,    0, ...,    0,    0,    0],
       ...,
       [5971,    8,   33, ...,    0,    0,    0],
       [   3,   22,  235, ...,    0,    0,    0],
       [   3,   14,  583, ...,    0,    0,    0]])>, 'decoder_inputs': <tf.Tensor: shape=(512, 20), dtype=int64, numpy=
array([[   2,    2,   89, ...,    0,    0,    0],
       [   2,    2,   20, ...,    0,    0,    0],
       [   2,    2,  435, ...,    0,    0,    0],
       ...,
       [   2,    2, 8891, ...,    0,    0,    0],
       [   2,    2,    7, ...,    0,    0,    0],
       [   2,    2,  384, ...,    0,    0,    0]])>}, <tf.Tensor: shape=(512, 20), dtype=int64, numpy=
array([[   2,   89,   22, ...,    0,    0,    0],
       [   2,   20, 1353, ...,    0,    0,    0],
       [   2,  435, 8450, ...,    0,    0,    0],
       ...,
       [   2, 8891,   